In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as BeautifulSoup

# Lista colori trovati
cafenoir_colors = []
cafenoir_colors_id = []

# categorie url 
url_cat = ['', 'uomo/'] 

for cat in url_cat:
    url = f'https://www.cafenoir.it/{cat}scarpe.html' 
    req = requests.get(url)

    if req.status_code != 200:
        if verbose:
            print(f'Page: {url} --> \033[41mStatus Code {req.status_code}\033[0m')
        break
                    
    soup = BeautifulSoup(req.text) 
    
    div_colors = soup.select('div.am-filter-items-colore_principale')
    
    # Estraggo colori
    color_elements = div_colors[0].select('div.swatch-option')
    for color in color_elements: 
        color_id = color['data-option-id']
        color_name = color['option-label']
        cafenoir_colors_id.append(color_id)
        cafenoir_colors.append(color_name)
            
# Stampo tutti i colori trovati
dic = { 'name' : cafenoir_colors, 'id' : cafenoir_colors_id }
cafenoir_colors_df = pd.DataFrame(dic)
cafenoir_colors_df = cafenoir_colors_df.groupby('name').id.min().to_frame()
cafenoir_colors_df

,id
name,
Arancio,309
Argento,291
Argento|Bianco,5157
Azzurro,1277
Azzurro|Bianco,5116
Beige,295
Beige|Bianco,5078
Bianco,296
Bianco|Beige,1081


In [2]:
verbose = True # abilita descrizione stato dello scraping

n = 1

# liste dati
prod_id = []
prod_category = []
prod_name = []
prod_price = []
prod_sales = []
prod_colors = []
prod_sex = []

for cat in url_cat:
    for color_name, color in cafenoir_colors_df.iterrows():
        for i in range(1, 10):
            url = f'https://www.cafenoir.it/{cat}scarpe.html?colore_principale={color.id}&p={i}'  
            req = requests.get(url)

            if req.status_code != 200:
                if verbose:
                    print(f'Page: {url} --> \033[41mStatus Code {req.status_code}\033[0m')
                break
            
            soup = BeautifulSoup(req.text)
            
            div_prod = soup.select('li.item.product')
    
            if len(div_prod) == 0: # break in caso non ci sono più prodotti
                break;
                
            # Estraggo prodotti nella pagina
            for prod in div_prod:
                # Estrazione dell'ID del prodotto
                prod_id.append(prod.find('a')['data-id'])
                # Estrazione del nome del prodotto
                prod_name.append(prod.find('a', class_='product-item-link').get_text(strip=True)) 
                # parametro strip per eliminare eventuali spazi all'inizio e alla fine 
                
                prod_category.append(prod_name[-1].lower().split(' ')[0].capitalize())
        
                # Estraggo prezzo del prodotto
                try:
                    prod_price.append(float(prod.find('span', class_='old-price').find('span', class_='price').get_text(strip=True).replace('\xa0€','').replace(',', '.')))                    
                    prod_sales.append(float(prod.find('span', class_='normal-price').find('span', class_='price').get_text(strip=True).replace('\xa0€','').replace(',', '.')))
                except:
                    prod_sales.append(None)
                    prod_price.append(float(prod.find('span', class_='normal-price').find('span', class_='price').get_text(strip=True).replace('\xa0€','').replace(',', '.')))                    
                    
                prod_sex.append('Donna' if cat=='' else 'Uomo')          
                prod_colors.append(color_name)
        
                # Stampo dei dati del prodotto
                if verbose:
                    print(n, prod_id[-1], prod_category[-1], prod_name[-1], prod_price[-1], prod_sales[-1], prod_colors[-1],  sep='\t')
                    n+=1
            if verbose:
                print(f'Page: {url} --> \033[42mStatus OK\033[0m')

1	C1FB6010R003	Zoccoli	zoccoli con tacco basso e fascia in crosta arancio	79.0	39.5	Arancio
2	C1HN9004Z011	Sandali	sandali doppia fascia in strass zeppa in rafia ambra	119.9	83.93	Arancio
3	C1GD9130M078	Sandali	sandali bassi modello ciabatta in sughero e strass bicolor arancio/marrone	69.0	34.5	Arancio
4	C1NB4030R003	Dècollète	dècollète  in pelle con listini arancio	129.0	64.5	Arancio
5	C1GD9001R003	Sandali	sandali con multilistini e strass arancio	89.9	62.93	Arancio
6	C1EF5014R003	Dècollète	dècollète in suede con punta affusolata e catena decorativa arancio	139.0	69.5	Arancio
7	C1FD8004R003	Sandali	sandali con fascia in cavallino maculato e tacco a clessidra arancio	119.0	59.5	Arancio
8	C1FD6002R003	Sandali	sandali in pelle scamosciata con fascia decorara e fiore in 3d. arancio	109.0	54.5	Arancio
Page: https://www.cafenoir.it/scarpe.html?colore_principale=309&p=1 --> Status OK
9	C1XO5557L001	Sandali	sandali doppia fascia effetto laminato argento	139.0	None	Argento
10	C1NA2027L001	Sand

In [3]:
# creazione del dataframe
cafe_dict = {'id': prod_id, 'category': prod_category, 'name': prod_name, 'price': prod_price, 'sales_price' : prod_sales, 'color':prod_colors, 'sex': prod_sex} 
cafe_df = pd.DataFrame(cafe_dict)

# replace delle categorie con typo errate in base a quelle di nero giardini
cafe_df.category = cafe_df.category.replace('Décollété', 'Décolleté')
cafe_df.category = cafe_df.category.replace('Dècollète', 'Décolleté')
cafe_df.category = cafe_df.category.replace('Dècollète\'', 'Décolleté')
cafe_df.category = cafe_df.category.replace('Ballerina', 'Ballerine')
cafe_df.category = cafe_df.category.replace('Tronchetto', 'Tronchetti')
cafe_df.category = cafe_df.category.replace('Ciabatta', 'Ciabatte')
cafe_df.category = cafe_df.category.replace('Mocassino', 'Mocassini')
cafe_df.category = cafe_df.category.replace('Francesina', 'Francesine')
cafe_df.category = cafe_df.category.replace('Stivaletto', 'Stivaletti')
cafe_df.category = cafe_df.category.replace('Stivale', 'Stivali')
cafe_df.category = cafe_df.category.replace('Sandalo', 'Sandali')
cafe_df.category = cafe_df.category.replace('Sneaker', 'Sneakers')
cafe_df.category = cafe_df.category.replace('Scarpa', 'Altro')
cafe_df.category = cafe_df.category.replace('Scarpe', 'Altro')

cafe_df = cafe_df.drop_duplicates(subset=['name', 'color'])
cafe_df.to_csv('cafenoir_scrap_260225.csv', index=False)
cafe_df

,id,category,name,price,sales_price,color,sex
0,C1FB6010R003,Zoccoli,zoccoli con tacco basso e fascia in crosta ara...,79.0,39.50,Arancio,Donna
1,C1HN9004Z011,Sandali,sandali doppia fascia in strass zeppa in rafia...,119.9,83.93,Arancio,Donna
2,C1GD9130M078,Sandali,sandali bassi modello ciabatta in sughero e st...,69.0,34.50,Arancio,Donna
3,C1NB4030R003,Décolleté,dècollète in pelle con listini arancio,129.0,64.50,Arancio,Donna
4,C1GD9001R003,Sandali,sandali con multilistini e strass arancio,89.9,62.93,Arancio,Donna
...,...,...,...,...,...,...,...
875,C1XN1017M005,Sneakers,sneakers uomo in pelle scamosciata taupe,179.0,71.60,Taupe,Uomo
876,C1XM6006M005,Mocassini,mocassini uomo in pelle scamosciata taupe,139.0,55.60,Taupe,Uomo
877,C1PC6434G046,Sneakers,sneakers da uomo in pelle scamosciata e tessut...,89.9,35.96,Verde,Uomo
878,C1TR6112G035,Mocassini,mocassini in pelle scamosciata con fiocco verd...,89.9,35.96,Verde,Uomo
